<a href="https://colab.research.google.com/github/champ-rtu/CE888/blob/main/Lap4/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

# **Load jester-data-1**

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv", header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [3]:
df = data.copy()
arr = df.values
arr.shape

(24983, 101)

In [4]:
data.iloc[6893,10]

1.41

In [ ]:
sum(data[])

In [25]:
rating = np.where(mat!=99)
rating[0]

array([    0,     0,     0, ..., 24982, 24982, 24982])

In [26]:
print(len(rating[0]), rating[1].shape)

1835357 (1835357,)


In [27]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [28]:
new_arr, idx = replace(arr, 0.1)

In [29]:
arr[idx[0][0], idx[1][0]]

3.45

In [30]:

new_arr[idx[0][0], idx[1][0]]

99.0

In [40]:
df.shape

(24983, 101)

In [28]:
#setting latent factor as 2 
n_latent_factors = 2

user_ratings = df.values

# Create set of random value
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [39]:
user_rating.shape[1]

101

In [31]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=1000, alpha=0.0001):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse1= []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
          mse1.append(mse)
    return mse1

In [33]:
pred = sgd(iterations=10, alpha=0.0001)

In [34]:
pred

[34.09019860828681,
 25.86787228396349,
 24.747125132356043,
 24.660924396967772,
 24.651273333442006,
 24.64943693950615,
 24.648690244229464,
 24.648103215129005,
 24.64755353875633,
 24.647026075529432]

In [35]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[7.48399630e+01, 8.59494133e-01, 2.62057129e-01, ...,
        7.42971667e-01, 3.50528855e-02, 1.11804890e+00],
       [9.63598687e+01, 1.05756622e+00, 3.66046183e-01, ...,
        9.82307338e-01, 4.82659902e-02, 1.47606634e+00],
       [5.45056318e+01, 6.93777605e-01, 1.51283811e-01, ...,
        5.05592212e-01, 2.11983078e-02, 7.63793610e-01],
       ...,
       [3.81003779e+01, 4.60328987e-01, 1.20124835e-01, ...,
        3.66317544e-01, 1.63911216e-02, 5.52241313e-01],
       [3.75327342e+01, 4.63067326e-01, 1.12735059e-01, ...,
        3.55834449e-01, 1.55340607e-02, 5.36870269e-01],
       [7.45837281e+01, 8.07110801e-01, 2.90010851e-01, ...,
        7.66318492e-01, 3.80902319e-02, 1.15102291e+00]])

In [71]:
predictions.shape

(24983, 101)

In [36]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [37]:
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 74.83996300636724)","(-7.82, 0.8594941334542137)","(8.79, 0.26205712855194696)","(-9.66, 0.31800672451581036)","(-8.16, -1.164480122853947)","(-7.52, 0.40548408425118065)","(-8.5, 1.4184451003598126)","(-9.85, -0.33456261463741566)","(4.17, -0.5030343878096517)","(-8.98, -0.4594829609638911)","(-4.76, 1.188658569606659)","(-8.5, 1.6895670688200595)","(-6.75, 1.3990661069291037)","(-7.18, -1.6550075883551074)","(8.45, 1.3367361926552723)","(-7.18, -1.619232281324875)","(-7.52, -2.9298387263721453)","(-7.43, -1.0030618164032983)","(-9.81, -0.5957554118596913)","(-9.85, 0.1729270582522909)","(-9.85, -0.9067403933900116)","(-9.37, 2.085639129837063)","(1.5, 0.8614505476518242)","(-4.37, 0.15208085810878805)","(-9.81, -1.4187899780828441)","(-8.5, 0.40905469213756646)","(1.12, 1.2346372416914404)","(7.82, 3.066771273838887)","(2.86, 1.4272883574601378)","(9.13, 2.9139729119344278)","(-7.43, -0.3936231964832416)","(2.14, 2.158530964778527)","(-4.08, 3.045986479003921)","(-9.08, -1.144369509835867)","(7.82, 0.8812611898780569)","(5.05, 2.913532820111724)","(4.95, 3.196648385661175)","(-9.17, -1.14376356764157)","(-8.4, 1.2431621639853263)","(-8.4, 1.0686006233986687)",...,"(8.59, 2.3820231035801727)","(3.59, 2.859728549309267)","(-6.84, 0.23493078137451115)","(-9.03, -0.5311062902532016)","(2.82, 2.2594853015187026)","(-1.36, 2.460245332645124)","(-9.08, -0.6836368021440873)","(8.3, 2.538638120168322)","(5.68, 2.541855748105661)","(-4.81, 0.5077269730311982)","(99.0, -0.48677487776149014)","(99.0, 2.238106137296352)","(99.0, 0.8970071999221211)","(99.0, -1.1340625487427056)","(99.0, -0.13875470947415317)","(99.0, 2.041309950836827)","(99.0, 0.639520505935173)","(-9.42, 1.4469215773307442)","(99.0, 0.17366251322547616)","(99.0, 0.9795518206502449)","(99.0, 1.567434085479249)","(-7.72, 0.7972549583097517)","(99.0, 1.692589030326605)","(99.0, 0.6207387336948388)","(99.0, 0.7504364425749196)","(99.0, 0.3101084352284196)","(99.0, 1.5698752776378118)","(99.0, 1.761445154459563)","(99.0, 2.8773065650628165)","(99.0, 0.5791451796226557)","(2.82, 1.6844310717121873)","(99.0, 1.013718814800244)","(99.0, 2.0676618088092127)","(99.0, 0.995299017042995)","(99.0, 0.9028362833553618)","(99.0, 1.2487986985230826)","(-5.63, 1.392618755160138)","(99.0, 0.7429716667574406)","(99.0, 0.03505288553290598)","(99.0, 1.1180488992540043)"
1,"(100.0, 96.35986868573764)","(4.08, 1.0575662241971535)","(-0.29, 0.3660461829448663)","(6.36, 0.4277834375526444)","(4.37, -1.5403941312078648)","(-2.38, 0.6142544172790947)","(-9.66, 1.800761314575295)","(-0.73, -0.418434122337652)","(-5.34, -0.6581900502351151)","(8.88, -0.6897167660120046)","(9.22, 1.494131364556561)","(6.75, 2.1318658008144893)","(8.64, 1.7865074924894553)","(4.42, -2.18784206899158)","(7.43, 1.7180083084088535)","(4.56, -2.1211487780761002)","(-0.97, -3.7107915989058524)","(4.66, -1.312792377661872)","(-0.68, -0.77996761574418)","(3.3, 0.24080391552469665)","(-1.21, -1.1975155846360221)","(0.87, 2.730028705566106)","(8.64, 1.1033810987067947)","(8.35, 0.15570206078512394)","(9.17, -1.8569492244001529)","(0.05, 0.48732789761709056)","(7.57, 1.5854384071172802)","(4.71, 3.9536620633865454)","(0.87, 1.8713855374595927)","(-0.39, 3.7136099358832766)","(6.99, -0.472097341211899)","(6.5, 2.7700966768965265)","(-0.92, 3.9216954200104763)","(7.14, -1.4512251826784133)","(9.03, 1.147398286875546)","(-1.8, 3.778248990341664)","(0.73, 4.094406716047287)","(7.09, -1.5158782025273951)","(3.4, 1.5563893198585517)","(-0.87, 1.368451538054632)",...,"(-6.7, 3.068065059587695)","(-3.35, 3.72305106302294)","(-9.03, 0.26252135732173115)","(4.47, -0.6444634056119048)","(4.08, 2.9165475647525603)","(-3.83, 3.151377321860689)","(8.74, -0.9525385709470355)","(1.12, 3.27929274585689

# **Load movies dataset**

In [83]:
it = pd.read_excel("https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/movies_latent_factors.xlsx", sheet_name='Items')
it.tail()

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
95,9806,The Incredibles (2004),0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,10020,Beauty and the Beast (1991),1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,36657,X-Men (2000),0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,36658,X2: X-Men United (2003),1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130
99,36955,True Lies (1994),1.734008,0.265436,-0.003391,-0.491754,-0.322739,0.850165,-0.150377,-0.075895,1.373386,0.934767,0.489299,-0.062678,0.333703,-0.223382,-0.022599


In [84]:
it['Factor11'].idxmin

<bound method Series.idxmin of 0    -0.973915
1    -0.561948
2     0.633667
3     0.526803
4     1.394326
        ...   
95   -0.846044
96   -0.862833
97   -0.163103
98   -0.291422
99    0.489299
Name: Factor11, Length: 100, dtype: float64>

In [100]:
itt = it.copy()

In [85]:
it.shape

(100, 17)

In [86]:
d = it['Factor11'].values

In [87]:
min(d)

-2.243340340044728

In [88]:
user = pd.read_excel("https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/movies_latent_factors.xlsx", sheet_name='Users',header=None)
user.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
1,4768,-0.204024,0.161079,-0.0904475,0.138495,-0.162934,0.163894,0.0515018,-0.0885822,0.126829,0.0659671,0.0850081,0.355404,0.007108,-0.118663,-0.0391254
2,156,-0.189652,-0.178979,-0.0914896,-0.00082278,-0.0326459,0.177209,-0.0981229,-0.0682826,-0.0115749,0.120866,-0.00993143,-0.0486062,0.0459162,0.113671,0.179873
3,5323,-0.115308,-0.0908861,-0.0531285,0.0184716,-0.0680808,-0.00482837,0.113005,0.102107,0.0347577,0.000692586,-0.0737118,-0.0194599,0.108372,0.0544706,-0.109552
4,174,-0.227462,-0.272532,-0.0172309,0.0543242,0.214755,-0.0726388,-0.0331216,-0.0865082,-0.131479,0.180403,0.0958896,-0.0823963,0.0367672,-0.165438,0.050692


In [89]:
user.shape

(26, 16)

In [90]:
its = it.values
us = user.values

In [91]:
its.shape

(100, 17)

In [92]:
us.shape[1]

16